In [ ]:
# default_exp navigator

In [ ]:
#hide
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#hide
!pip install nbdev
!pip install fastcore

In [ ]:
#hide
% cd /content/drive/My\ Drive/fa_convnav

/content/drive/My Drive/fa_convnav


In [ ]:
#hide
#not deps but we need them to use nbdev and run tests
from nbdev import * 
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#hide
! pip install fastai2

In [ ]:
#hide
from fastai2.basics import *
from fastai2.callback.all import *
from fastai2.vision.all import *
from torch import torch

pets = DataBlock(blocks=(ImageBlock, CategoryBlock), 
                 get_items=get_image_files, 
                 splitter=RandomSplitter(),
                 get_y=RegexLabeller(pat = r'/([^/]+)_\d+.jpg$'),
                 item_tfms=Resize(460),
                 batch_tfms=[*aug_transforms(size=224, max_rotate=30, min_scale=0.75), Normalize.from_stats(*imagenet_stats)])

dls = pets.dataloaders(untar_data(URLs.PETS)/"images",  bs=128)

In [ ]:
#hide
learn = cnn_learner(
    dls, 
    densenet121, 
    opt_func=partial(Adam, lr=slice(3e-3), wd=0.01, eps=1e-8), 
    metrics=error_rate, 
    config=cnn_config(ps=0.33)).to_fp16()

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 63.6MB/s]


# Navigator

> CNN viewer and navigator

In [ ]:
#export
from fa_convnav.models import models
from dataclasses import dataclass
from pandas import DataFrame, option_context


In [ ]:
#export
def get_row(l, m):
  "Construct a convnav dataframe row from information in fastai `learner.named_modules()`"

  # create generic row data from `l` (model.named_module() layer) and `m` (model_type)
  lyr_name = l[0]
  lyr_obj = l[1]
  ln_split = str(lyr_name).split('.', 4)
  ln_n_splits = len(ln_split)
  lyr_str = str(lyr_obj)

  tch_cls_str = str(type(lyr_obj))
  tch_cls_substr =  tch_cls_str[tch_cls_str.find("<class")+8: tch_cls_str.find(">")-1]
  tch_cls = tch_cls_substr.split('.')[-1]

  div = tch_cls if lyr_name == '0' or lyr_name == '1' else ''
  mod = tch_cls if ln_n_splits == 2 else ''
  blk = tch_cls if ln_n_splits == 3 and not lyr_name.startswith('1') else ''
  lyr = lyr_str[:90]

  # customise generic row for peculiarities of specific models
  if m == 'vgg' or m == 'alexnet':
    if ln_n_splits >2: ln_split[2] = '' 
    blk = ''

  elif m == 'squeezenet':
     blk = tch_cls if ln_n_splits == 3 and tch_cls == 'Fire' else '' 
     if blk == 'Fire': lyr = ''   

  elif m == 'resnet':
    if blk == 'BasicBlock' or blk == 'Bottleneck': lyr = ''
    else:
      if ln_n_splits > 4: lyr = f". {lyr_str[:87]}" 
      if ln_n_splits == 4 and ln_split[3] == 'downsample': lyr = tch_cls
       
  elif m == 'densenet':
    lyr_name = lyr_name.replace('denseblock', '').replace('denselayer', '')
    ln_split = str(lyr_name).split('.', 4)
    mod = tch_cls if (lyr_name.startswith('0') and ln_n_splits == 3) or (lyr_name.startswith('1') and ln_n_splits == 2) else ''
    blk = tch_cls if ln_n_splits == 4 and tch_cls == '_DenseLayer' else '' 
    if mod == '_DenseBlock' or mod == '_Transition' or blk == '_DenseLayer': 
      lyr = ''
    else: 
      if lyr_name == '0' or lyr_name == '1': div = tch_cls 
      if lyr_name == '0.0': div = f'. {tch_cls}'
      
  elif m == 'xresnet':
    blk = tch_cls if ln_n_splits == 3 and tch_cls == 'ResBlock' else '' 
    if mod == 'ConvLayer' or blk == 'ResBlock': 
      lyr = ''
    else:
      if ln_n_splits < 4: lyr =  lyr_str[:90]
      elif ln_n_splits == 4 and tch_cls == 'Sequential': lyr =  tch_cls
      elif ln_n_splits == 4 and tch_cls == 'ReLU': lyr =  lyr_str[:90]
      elif ln_n_splits == 5 and tch_cls == 'ConvLayer': lyr =  f'. {tch_cls}'
      else: lyr =  f'. . {lyr_str[:82]}'

  else:
    raise Exception("Model type not recognised")
  
  return {
      'Layer_name': lyr_name, 
      'Model': tch_cls if lyr_name == '' else '',
      'Division': div,
      'Module': mod,
      'Block': blk, 
      'Layer_description': lyr,
      'Torch_class': tch_cls_substr,
      'Output_dimensions': '',
      'Parameters': '',
      'Trainable': '',
      'Currently': '',
      'div_id': ln_split[0] if ln_n_splits >0 else '',  
      'mod_id': ln_split[1] if ln_n_splits >1 else '',  
      'blk_id': ln_split[2] if ln_n_splits >2 else '',
      'lyr_id': ln_split[3] if ln_n_splits >3 else '',
      'tch_cls': tch_cls,
      'out_dim': '',
      'current': '',
      'lyr_blk': '', 
      'lyr_mod': '',
      'blk_mod': '',
      'lyr_obj': lyr_obj
      }

In [ ]:
#export
def find_model(n): 
    "Returns tuple of model type and name (e.g. ('resnet', 'resnet50')) given `n`, the number of named_modules in `learn.model.named_modules()`"
    for d in models:
      match = [(k, m) for k, v in d.items() for m, l in v if l == n]
      if match != []: break
    if len(match) > 0: return match[0] # (model_type, model_name)
    assert True, 'Model not supported. Use `supported_models()` to get a list of supported models.'

In [ ]:
#export
@dataclass
class CNDF:
  "Compile information from fastai `learner` and 'layer_info(learner)` into a dataframe"
  learner: any
  layer_info: tuple

  def sz_tn(t):
    "Adds batch size `bs` to a list if layer dimensions `t`"
    return [self.bs if i==0 else s for i, s in enumerate(t)]

  def __post_init__(self):
    self.model = self.learner.model                                         # fastai `learner.model` object
    self.layers = list(self.learner.model.named_modules())                  # fastai `named_modules` method
    self.num_layers = len(self.layers)                    
    self.model_type, self.model_name = find_model(self.num_layers)

    inp, info = self.layer_info                                   
    self.inp_sz = [sz for sz in inp[0].shape]                          # `inp_sz` =  (bs, ch, h, w)
    self.bs = self.inp_sz[0]                               

    # create base dataframe `df` from a list of formatted rows in `layers`
    df = DataFrame([get_row(l, self.model_type) for l in self.layers]) 

    # remove layer descriptions from container rows
    df.at[0, 'Layer_description'] = ''
    df.loc[(df['Division'].str.contains('Sequential')) | \
          (df['Module'] == 'Sequential') | \
          (df['Module'] == 'AdaptiveConcatPool2d'), 'Layer_description'] = ''

    # add layer info (`o`, `p`, `t`, and `Frozen`/`Unfrozen`) from layer_info to all non-container rows
    info_gen = ((_, p, t, self.add_bs(o)) for _, p, t, o in info) # info_gen = generator
    for row in df.itertuples(): 
      idx = row.Index
      if row.Layer_description not in ['', '. ConvLayer', 'Sequential']:
        _, p, t, o = next(info_gen)
        df.at[idx, 'Output_dimensions'] = str(o)
        df.at[idx, 'Parameters'] =  p
        df.at[idx, 'Trainable'] = t
        if 'Conv2d' in row.Torch_class:
          df.at[idx, 'Currently'] = 'Unfrozen' if t else 'Frozen'

    # backfill container rows with summary layer information and layer/block counts
    # 1.set up index stores and counters
    m, b  = 0, 0                              
    layer_count = [0, 0, 0]                                       # layers in [divs, modules, blocks]
    block_count = [0, 0]                                          # blocks in [divs, modules]
    frozen_count=[[0,0], [0, 0], [0,0]]                           # [Frozen, Unfrozen] layers in [divs, modules, blocks],  

    # 2.iterate over rows, incrementing counters with each new row
    for row in df.itertuples():
      idx = row.Index
      if row.Currently == 'Frozen':
        for i in [0,1,2]: frozen_count[i][0] += 1 
      if row.Currently == 'Unfrozen':
        for i in [0,1,2]: frozen_count[i][1] += 1
      if row.Layer_description != '':
        for i in [0,1,2]: layer_count[i] += 1 

      # backfill 'Module' container rows with layer_info and block and layer counts
      if (row.Output_dimensions == '' and row.Module != '') or row.Layer_name == '1':
        m = idx if m == 0 else m
        df.at[m, 'out_dim'] = df.at[idx-1, 'Output_dimensions']
        df.at[m, ['current', 'blk_mod', 'lyr_mod']] = self.get_frozen(frozen_count[1]), block_count[1], layer_count[1] 
        m = idx
        layer_count[1] = block_count[1] = 0
        for i in [0, 1]: frozen_count[1][i] = 0

      # backfill 'Block' container rows with layer_info and layer counts
      if (row.Output_dimensions == '' and row.Block != '') or row.Layer_name == '1':
        b = idx if b == 0 else b
        df.at[b, 'out_dim'] = df.at[idx-1, 'Output_dimensions'] or df.at[idx-2, 'Output_dimensions']  
        df.at[b, ['current', 'lyr_blk']] = self.get_frozen(frozen_count[2]), layer_count[2]
        b = idx
        layer_count[2] = 0
        for i in [0, 1]: block_count[i] += 1
        for i in [0, 1]: frozen_count[2][i] = 0
      
    # 3.backfill division container rows with summary layer_info and block and layer counts
    div0_idx = df[df['Layer_name'] == '0'].index.tolist()[0]
    div1_idx = df[df['Layer_name'] == '1'].index.tolist()[0] 

    df.at[div0_idx, 'out_dim'] = df.at[div1_idx-1, 'Output_dimensions'] or df.at[div1_idx-2, 'Output_dimensions']  
    df.at[div0_idx, ['current', 'lyr_mod', 'blk_mod']] = self.get_frozen(frozen_count[0]), layer_count[0], block_count[0]

    df.at[div1_idx, 'out_dim'] = df.iloc[-1]['Output_dimensions']

    self._cndf = df

  
  def add_bs(self, dims):
    "Adds batch size `bs` to a list if layer dimensions `dims`"
    return [self.bs if i==0 else s for i, s in enumerate(dims)]

  @staticmethod
  def get_frozen(f):
    "Returns a string interpretation of the number of frozen/unfrozen layers in tuple `f`"
    if f[0] == 0: return 'Unfrozen'
    elif f[1] == 0: return 'Frozen'
    else: return f'{f[0]}/{(f[0]+f[1])} layers frozen'

  @property
  def output_dimensions(self):
    "Returns output dimensions of model (bs, classes)"
    return self._cndf.iloc[-1]['Output_dimensions']

  @property
  def frozen_to(self):
    "Returns parameter group model is curently frozen to"
    return self.learner.opt.frozen_idx + 1

  @property
  def num_param_groups(self):
    "Returns number of parameter groups"
    return len(self.learner.opt.param_groups)

  @property
  def batch_size(self):
    "Returns the batch size of the current learner"
    return self.bs

  @property
  def input_sizes(self):
    "Returns the sizes (dimensions bs, ch, h, w) of the model)"
    return self.input_sz

  @property
  def model_info(self):
    "Return an info string derived from`learn.model`"  
    res = f"{self.model_type.capitalize()}: {self.model_name.capitalize()}\n"
    res += f"Input shape: {self.inp_sz} (bs, ch, h, w)\n"
    res += f"Output features: {self.output_dimensions} (bs, classes)\n" 
    res += f"Currently frozen to parameter group {self.frozen_to} out of {self.num_param_groups}" 
    return res

  @property
  def cndf(self):
    "Returns a ConvNav dataframe"
    return self._cndf.copy()

  @staticmethod
  def supported_models():
    "Prints list of supported models from 'models' list (imported from models)"
    print('Supported models')
    print('================\n')
    for d in models:
        [[print(m) for m, l in v] for k, v in d.items()]

CNDF class builds a dataframe with information froma fastai learner and fastai layer_info(learner) mathod. The resulting dataframe, called a CNDF or ConvNav dataframe, represents both the structure and information of the original model and architecture. 

Create a new CNDF dataframe with 

```
cndf = CNDF(learner, layer_info(learner)
```

A CNDF dataframe is automatically created when a ConvNav instance is created and all the following methods are also available from within a ConvNav instance. 


In [ ]:
show_doc(CNDF.supported_models)

<h4 id="CNDF.supported_models" class="doc_header"><code>CNDF.supported_models</code><a href="__main__.py#L138" class="source_link" style="float:right">[source]</a></h4>

> <code>CNDF.supported_models</code>()

Prints list of supported models from 'models' list (imported from models)

In [ ]:
show_doc(CNDF.cndf)

<h4 id="CNDF.cndf" class="doc_header"><code>CNDF.cndf</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Returns a ConvNav dataframe

In [ ]:
show_doc(CNDF.batch_size)

<h4 id="CNDF.batch_size" class="doc_header"><code>CNDF.batch_size</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Returns the batch size of the current learner

In [ ]:
show_doc(CNDF.input_sizes)

<h4 id="CNDF.input_sizes" class="doc_header"><code>CNDF.input_sizes</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Returns the sizes (dimensions bs, ch, h, w) of the model)

In [ ]:
show_doc(CNDF.output_dimensions)

<h4 id="CNDF.output_dimensions" class="doc_header"><code>CNDF.output_dimensions</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Returns output dimensions of model (bs, classes)

In [ ]:
show_doc(CNDF.model_info)

<h4 id="CNDF.model_info" class="doc_header"><code>CNDF.model_info</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Return an info string derived from`learn.model`

In [ ]:
#export
class CNDFView:
  "Class to view a convnav dataframe"

  def copy_layerinfo(self, df):
    "Copy layer information and block/layer counts over from silent columns to displayed columns"
    df.loc[df['Division'] == '', 'Division'] = df['div_id']
    df.loc[df['Module'] == '', 'Module'] = df['mod_id']
    df.loc[df['Block'] == '', 'Block'] = df['blk_id']
    df.loc[df['Output_dimensions'] == '', 'Output_dimensions'] = df['out_dim']
    df.loc[df['Currently'] == '', 'Currently'] = df['current']
    return df

  def view(self, df=None, verbose=3, tight=True, truncate=0, align_cols='left', top=False, return_df=False):
    "Display dataframe `df` with options and styling"

    _df = df if df is not None else self._cndf.copy()

    assert type(_df) == DataFrame and 'Layer_name' in _df.columns, 'Not a valid convnav dataframe'
    assert isinstance(truncate, int) and -10 <= truncate <= 10, f"Argument 'truncate' must be an integer between -10 (show more cols) and +10 (show fewer cols)"
    assert isinstance(verbose, int) and 1 <= verbose <= 4, f"Argument verbose must be 1 2 or 3 "
    if not isinstance(tight, bool): tight = True

    if len(_df) < 10: tight = False
    if verbose != 3: truncate = (10, 4, 0, -10)[verbose-1]

    post_msg = ''
    if top and len(_df) > 10:
      post_msg = f'...{len(_df)-10} more layers'
      _df = _df.iloc[:10]
      tight=False
      
    _df.index.name = 'Index'

    if len(_df) == 0:
      print('No data to display')
      return None

    with option_context("display.max_rows", 1000): 
      _df_styled = _df.iloc[:,:-(11+truncate)].style.set_properties(**{'text-align': align_cols})
      if tight: 
        display(_df_styled)
      else:
        display(_df.iloc[:,:-(11+truncate)]) 
    print(post_msg)

    if return_df: return(_df)

  def copy_view(self, df, **kwargs):
    "Copy over layer information then call `view` to display dataframe"
    df = self.copy_layerinfo(df)
    self.view(df=df, **kwargs)

In [ ]:
show_doc(CNDFView.view)

<h4 id="CNDFView.view" class="doc_header"><code>CNDFView.view</code><a href="__main__.py#L13" class="source_link" style="float:right">[source]</a></h4>

> <code>CNDFView.view</code>(**`df`**=*`None`*, **`verbose`**=*`3`*, **`tight`**=*`True`*, **`truncate`**=*`0`*, **`align_cols`**=*`'left'`*, **`top`**=*`False`*, **`return_df`**=*`False`*)

Display dataframe `df` with options and styling

Method to format and display a ConvNav dataframe. Defaults to displaying the instance dataframe built by CNDF but will accept a valid ConvNav dataframe `df` as an argument. Other arguments:


*   `verbose`: 1=Index and Layer_name columns only; 2=Model structure; 3=Model Structure and layer_info (output dims, params and frozen/unfrozen) (default); 4=includes normally hidden columns.  
*   `tight`: False=normal row spacing; True=tight layout with minimal space between rows (best for large models with many rows to display). The default is True but dfs with fewer than 10 rows are automatically displayed with normal spacing.
*   `truncate`: truncate number displayed columns by integer value between -10 and 10. Default=0. Negative values reveal hidden columns. Overidden when the verbose argument is set to a non-default setting.
*   `align_cols`: Left (default); right
*   `top`: like the pandas head method, displays first 10 rows only followed by a count of undisplayed rows
*   `return_df`: return the formatted df to the caller if True. False returns None (default). 




CNDFView methods are also accessible from Convnav instances. 

```
cn = ConvNav(learner, layer_info(learner)
cn.view()
```




In [ ]:
#export
class CNDFSearch:
  "Class to search a convnav dataframe, display the results in a dataframe and return the matching layer objects"

  def find_layer(self, df, searchterm, exact):
    "Searches `df` for `searchterm`, returning exact matches only if `exact=True` otherwise any match"

    if isinstance(searchterm, int):
      assert searchterm >= 0 and searchterm <= len(self._cndf), f'Layer ID out of range: min 0, max {len(df)}'
      x = df.iloc[searchterm].copy()
      x = DataFrame(x).transpose()
      return x    
      
    if isinstance(searchterm, float): searchterm = str(searchterm)
      
    if isinstance(searchterm, dict):
      for col, s in searchterm.items():
        assert col in df.columns, f'{col} not a valid column identifier. Valid column names are {df.columns}'
        return df[df[col] == s].copy() if exact else df[df[col].str.contains(s)].copy()
      return x

    if isinstance(searchterm, str):
      searchterm = searchterm.strip(' \.')
      cols = {'Layer_name', 'Torch_class', 'Division', 'Module', 'Block', 'Layer_description'}
      if exact: 
        for col in cols:
          x = df[df[col] == searchterm].copy()
          if not x.empty: return x
      else: 
        for col in cols:
          x = df[df[col].str.contains(searchterm)].copy()
          if not x.empty: return x
      return x
         
    assert True, 'Unrecognizable searchterm'
        
  def search(self, searchterm, exact=True): 
    "Finds any single or combination of container(s), block(s) or layer(s) in the model_df"

    _df = copy(self._cndf)

    if isinstance(searchterm, float): searchterm = str(searchterm)

    if isinstance(searchterm, int): 
      _df = self.find_layer(_df, searchterm, True) 

    elif isinstance(searchterm, str): 
      _df = self.find_layer(_df, searchterm, exact)  

    elif isinstance(searchterm, dict):
      _df = DataFrame()  
      for col, s in searchterm.items():
        new_df = self.find_layer(self._cndf, {col:s}, exact)
        _df = pd.concat((_df, new_df), axis=0, ignore_index=False).drop_duplicates('Layer_name')

    elif isinstance(searchterm, list):
      _df = DataFrame()  
      for s in searchterm: 
        new_df = self.find_layer(self._cndf, s, exact)
        _df = pd.concat((_df, new_df), axis=0, ignore_index=False).drop_duplicates('Layer_name')

    elif isinstance(searchterm, tuple):
      for s in searchterm:
        _df = self.find_layer(_df, s, exact)

    else: assert True, 'Unrecognizable searchterm'

    if _df is not None and not _df.empty:
      print(f'{len(_df)} layers found matching searchterm(s): {searchterm}\n')
      self.view(df=_df)
      return _df['lyr_obj'].tolist()
    else: 
      print(f'No matches for searchterm(s): {searchterm}\n')
      return None

In [ ]:
show_doc(CNDFSearch.search)

<h4 id="CNDFSearch.search" class="doc_header"><code>CNDFSearch.search</code><a href="__main__.py#L36" class="source_link" style="float:right">[source]</a></h4>

> <code>CNDFSearch.search</code>(**`searchterm`**, **`exact`**=*`True`*)

Finds any single or combination of container(s), block(s) or layer(s) in the model_df

`Searchterm` can be:

*   `int` : the module with Index number `int` is returned
*   `float`: module(s) where `str(float)` matches the Layer_name are returned
*   `str`: module(s) with `str` in one of 'Layer_name', 'Torch_class', 'Division', 'Module', 'Block', 'Layer_description' are returned. Columns are searched in this order with the search ending with the first column to make a match/matches. 
*   `dict`, e.g. {'col', 'str'} matches for `str` in column `col` only are returned

Searchterms can also be a combined as follows:

*   `[101, 102, 105]` matches modules with the Index numbers 101, 102 and 105. Lists of floats, strings, dicts or combinations thereof are also valid searchterms.
*   `('0.5', 'conv2d')` matches modules where BOTH `0.5` AND `conv2d` match. Tuples of floats, strings, dicts or combinations thereof are also valid.
*   `{{'col1', 'str1'}, {'col2', 'str2'}}` matches `str1` in `col1` as well as `str2` in `col`'.

Kwarg exact=True (dafault) matches searchterms exactly with column entries, otherwise (exact=False) all column entries that contain the searchterm are matches. 



CNDFSearch methods are also accessible from Convnav instances.

```
cn = ConvNav(learner, layer_info(learner)
seachterm = 128
cn.search(searchterm)
```

In [ ]:
#export
class ConvNav(CNDF, CNDFSearch, CNDFView):
  "Class to view fastai supported CNNs, search and select module(s) and layer(s) for further investigation"
  def __init__(self, learner, layer_info):
    super().__init__(learner, layer_info )

  def __len__(self):
    return len(self._cndf)

  def __str__(self) -> str:
    return self.model_info

  def __call__(self):
    self.view(head=True)

  def __contains__(self, s):
    return self.search(s)

  @property
  def head(self):
    "Print `model` head summary info and layers"
    df = self._cndf.copy()
    df = df[df['Layer_name'].str.startswith('1')]
    if not df.empty:
      res = f"{self.model_type.capitalize()}: {self.model_name.capitalize()}\n"
      res += f"Input shape: {self._cndf.iloc[1]['out_dim']} (bs, filt, h, w)\n"
      res += f"Output features: {self.output_dimensions} (bs, classes)\n" 
      print(res)
      self.view(df, truncate=1)
    else:
      res = "Model has no head"
      print(res)

  @property
  def body(self):
    "Print `model` body summary info and layers"
    df = self._cndf.copy()
    df = df.loc[df['Layer_name'].str.startswith('0')]
    if not df.empty:
      res = f"{self.model_type.capitalize()}: {self.model_name.capitalize()}\n"
      res += f"Input shape: {self.input_sizes} (bs, ch, h, w)\n"
      res += f"Output dimensions: {df.iloc[-1]['Output_dimensions']} (bs, filt, h, w)\n"
      res += f"Currently frozen to parameter group {self.frozen_to} out of {self.num_param_groups}\n" 
      print(res)
      self.view(df)
    else:
      res = "Model body has no contents"
      print(res)

  @property
  def divs(self):
    "Print Summary information from `model` head and body"
    df = self._cndf[(self._cndf['Layer_name'] == '0') | (self._cndf['Layer_name'] == '1')].copy()

    for i in range(2):
      df_div = self._cndf.loc[self._cndf['div_id'] == str(i)].copy()
      df.iloc[i]['Model'] = self.model_name
      df.iloc[i]['Module'] = len(df_div[df_div['Module'] != ''])
      df.iloc[i]['Block'] = len(df_div[df_div['Block'] != ''])
      df.iloc[i]['Layer_description'] = len(df_div[df_div['Layer_description'] != ''])
      params = df_div['Parameters'].values
      params_summed = sum(filter(lambda i: isinstance(i, int), params))
      df.iloc[i]['Parameters'] = params_summed

    df['Output_dimensions'] = df['out_dim']
    df.iloc[0]['Currently'] = df.iloc[0]['current']

    df = df.rename(columns={'Module': 'Child modules', 'Block': 'Blocks', 'Layer_description': 'Layers'})
    print(f"{self.model_name.capitalize()}\nDivisions:  body (0), head (1)\n")
    self.view(df, tight=False)

  @property
  def dim_transitions(self):
    "Prints and returns all `model` layers which change dimensions between input and output"
    df = self._cndf[self._cndf['Torch_class'].str.contains('Conv2d')].copy()

    n = []
    old_dims = 0
    for i, row in enumerate(df.iterrows()):
      row=row[1]
      new_dims = row['Output_dimensions'].rstrip(']').split(',')[-1]
      if new_dims != old_dims:
        n.append(i)
        old_dims = new_dims
    df = df.iloc[n]

    print(f"{self.model_name.capitalize()}\nLayer dimension changes\n")
    self.copy_view(df, tight=False)
    return df['lyr_obj'].tolist()

  @property
  def linear_layers(self):
    "Prints and returns all linear layers in the `model`"
    df = self._cndf[self._cndf['Torch_class'].str.contains('Linear')].copy()
    df['Division'] = df['div_id']

    print(f"{self.model_name.capitalize()} linear layers\n")
    self.view(df, truncate=1, tight=False)
    return df['lyr_obj'].tolist()


In [ ]:
show_doc(ConvNav.head)

<h4 id="ConvNav.head" class="doc_header"><code>ConvNav.head</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Print `model` head summary info and layers

In [ ]:
show_doc(ConvNav.body)

<h4 id="ConvNav.body" class="doc_header"><code>ConvNav.body</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Print `model` body summary info and layers

In [ ]:
show_doc(ConvNav.divs)

<h4 id="ConvNav.divs" class="doc_header"><code>ConvNav.divs</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Print Summary information from `model` head and body

In [ ]:
show_doc(ConvNav.dim_transitions)

<h4 id="ConvNav.dim_transitions" class="doc_header"><code>ConvNav.dim_transitions</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Prints and returns all `model` layers which change dimensions between input and output

Finds conv layers in the model where activation dimensions change betwen input and output. Transition layers are often good points to investigate model function and `dim_transitions` returns the layer objects for attaching callbacks and hooks. 

In [ ]:
cn = ConvNav(learn, layer_info(learn))
layers = cn.dim_transitions

Densenet121
Layer dimension changes



,Layer_name,Model,Division,Module,Block,Layer_description,Torch_class,Output_dimensions,Parameters,Trainable,Currently
Index,,,,,,,,,,,
3,0.0.conv0,,0,Conv2d,conv0,"Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)",torch.nn.modules.conv.Conv2d,"[128, 64, 112, 112]",9408,False,Frozen
11,0.0.1.1.conv1,,0,0,1,"Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)",torch.nn.modules.conv.Conv2d,"[128, 128, 56, 56]",8192,False,Frozen
59,0.0.2.1.conv1,,0,0,2,"Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)",torch.nn.modules.conv.Conv2d,"[128, 128, 28, 28]",16384,False,Frozen
149,0.0.3.1.conv1,,0,0,3,"Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)",torch.nn.modules.conv.Conv2d,"[128, 128, 14, 14]",32768,False,Frozen
323,0.0.4.1.conv1,,0,0,4,"Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)",torch.nn.modules.conv.Conv2d,"[128, 128, 7, 7]",65536,False,Frozen


In [ ]:
show_doc(ConvNav.linear_layers)

<h4 id="ConvNav.linear_layers" class="doc_header"><code>ConvNav.linear_layers</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Prints and returns all linear layers in the `model`